<a href="https://colab.research.google.com/github/Nohyeong/ML-BO_membrane_design/blob/main/Li_Bayesian_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
!pip install rdkit
!pip install category_encoders
!pip install shap

Mounted at /gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 8.2 MB/s eta 0:00:00


In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import category_encoders as ce
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from hyperopt import hp, tpe, Trials, STATUS_OK, fmin
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, PowerTransformer, QuantileTransformer
import warnings
warnings.filterwarnings("ignore")

sc = PowerTransformer()

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error: Creating directory. ' + directory)

class morgan_fp:
    def __init__(self, radius, length):
        self.radius = radius
        self.length = length
    def __call__(self, smiles):
        mol = Chem.MolFromSmiles(smiles)
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, int(self.radius), int(self.length))
        npfp = np.array(list(fp.ToBitString())).astype('float32')
        return npfp

def conv_data_pd(data, data_t, fp, en, scaler, numeric_features, salt_per):
    data['a1-fp'] = data['a1-smile'].apply(fp)
    data['a2-fp'] = data['a2-smile'].apply(fp)

    data['a-fp'] = [x + y for x, y in zip(data['a1-fp'], data['a2-fp'])]
    data['b-fp'] = data['b-smile'].apply(fp)

    x_a=np.array(list(data['a-fp']))
    x_b=np.array(list(data['b-fp']))

    if salt_per == 'per':
      X_train = data_t.drop(columns=['Monomer A1 type', 'a1-smile','Monomer A2 type', 'a2-smile', 'b-smile', 'a1-fp', 'a2-fp', 'a-fp','b-fp']).copy()

    elif salt_per == 'rej':
      X_train = data_t.drop(columns=['Monomer A1 type', 'a1-smile','Monomer A2 type', 'a2-smile', 'b-smile', 'a1-fp', 'a2-fp', 'a-fp','b-fp']).copy()

    if salt_per == 'per':
      x = data.drop(columns=['Monomer A1 type', 'a1-smile','Monomer A2 type', 'a2-smile', 'b-smile', 'a1-fp', 'a2-fp', 'a-fp','b-fp']).copy()

    elif salt_per == 'rej':
      x = data.drop(columns=['Monomer A1 type', 'a1-smile','Monomer A2 type', 'a2-smile', 'b-smile', 'a1-fp', 'a2-fp', 'a-fp','b-fp']).copy()

    hh=en.fit_transform(X_train[categorical_features])
    # SC= scaler.fit(hh[numeric_features])
    columns_to_drop = ['a1-fp', 'a2-fp', 'a-fp', 'b-fp']
    x.drop(columns=[col for col in columns_to_drop if col in x.columns], inplace=True)

    col_pd = en.transform(x[categorical_features])
    xxxx = x[numeric_features]
    num_pd = pd.DataFrame(data= xxxx, columns=numeric_features) #pd
    xxxxx = np.concatenate((x_a, x_b), axis =1)
    fp_pd = pd.DataFrame(data= xxxxx, columns=[f'f_{i}' for i in range(2*x_a.shape[1])])

    x_pd = pd.concat([fp_pd, col_pd, num_pd], axis=1)
    x_pd.reset_index(drop=True, inplace=True)

    return x_pd

def conv_data_pd_opt(data, data_t, fp, en, scaler, numeric_features, salt_per):

    data['a1-fp'] = data['a1-smile'].apply(fp)
    data['a2-fp'] = data['a2-smile'].apply(fp)

    data['a-fp'] = [x + y for x, y in zip(data['a1-fp'], data['a2-fp'])]
    data['b-fp'] = data['b-smile'].apply(fp)

    x_a=np.array(list(data['a-fp']))
    x_b=np.array(list(data['b-fp']))

    if salt_per == 'per':
      X_train = data_t.drop(columns=['Monomer A1 type', 'a1-smile','Monomer A2 type', 'a2-smile', 'b-smile']).copy()

    elif salt_per == 'rej':
      X_train = data_t.drop(columns=['Monomer A1 type', 'a1-smile','Monomer A2 type', 'a2-smile', 'b-smile']).copy()

    if salt_per == 'per':
      x = data.drop(columns=['Monomer A1 type', 'a1-smile','Monomer A2 type', 'a2-smile', 'b-smile']).copy()

    elif salt_per == 'rej':
      x = data.drop(columns=['Monomer A1 type', 'a1-smile','Monomer A2 type', 'a2-smile', 'b-smile']).copy()

    hh=en.fit_transform(X_train[categorical_features])
    # SC= scaler.fit(hh[numeric_features])
    columns_to_drop = ['a1-fp', 'a2-fp', 'a-fp', 'b-fp']
    x.drop(columns=[col for col in columns_to_drop if col in x.columns], inplace=True)

    col_pd = en.transform(x[categorical_features])
    xxxx = x[numeric_features]
    num_pd = pd.DataFrame(data= xxxx, columns=numeric_features) #pd
    xxxxx = np.concatenate((x_a, x_b), axis =1)
    fp_pd = pd.DataFrame(data= xxxxx, columns=[f'f_{i}' for i in range(2*x_a.shape[1])])

    x_pd = pd.concat([fp_pd, col_pd, num_pd], axis=1)
    x_pd.reset_index(drop=True, inplace=True)

    return x_pd


def objective_per(params):
    global ITERATION
    ITERATION +=1
    for name in ['max_depth', 'n_estimators','fp_length']:
        params[name] = int(params[name])
    loss, train_loss = fit(params, train_data_per, numeric_features, 'per')
    loss =loss
    off_connection = open(out_file, 'a')
    writer = csv.writer(off_connection)
    writer.writerow([loss,train_loss, params, ITERATION])
    pickle.dump(bayes_trial, open("per_2_xgb.p", "wb"))
    return {'loss':loss,'train_loss':train_loss, 'params': params, 'iteration':ITERATION, 'status':STATUS_OK}

def objective_rej(params):
    global ITERATION
    ITERATION +=1
    for name in ['max_depth', 'n_estimators','fp_length']:
        params[name] = int(params[name])
    loss, train_loss = fit(params, train_data_rej, numeric_features, 'rej')
    loss =loss
    off_connection = open(out_file, 'a')
    writer = csv.writer(off_connection)
    writer.writerow([loss,train_loss, params, ITERATION])
    pickle.dump(bayes_trial, open("rej_1.p", "wb"))
    return {'loss':loss,'train_loss':train_loss, 'params': params, 'iteration':ITERATION, 'status':STATUS_OK}

def data_split(train_data_per, n_splits, test_size, random_state):
  sss = ShuffleSplit(n_splits=1, test_size=0.2,random_state=10)

  for train_index, test_index in sss.split(train_data_per):
      train_data = train_data_per.iloc[train_index]
      test = train_data_per.iloc[test_index]
      train_data.reset_index(drop=True, inplace=True)
      test.reset_index(drop=True, inplace=True)
      # train_data.to_csv('per_train_data.csv', index = False)
      # test.to_csv('per_test.csv', index = False)
  return train_data, test

fp_length_list = [2048]

space = {
         'max_delta_step': hp.uniform('max_delta_step', 0, 10),
         'learning_rate': hp.uniform('learning_rate', 0.001, 0.1),
         'max_depth': hp.quniform('max_depth', 1,10,1),
         'min_child_weight': hp.uniform('min_child_weight', 1,10),
         'subsample': hp.uniform('subsample', 0.5, 1.0),
         'reg_alpha':hp.uniform('reg_alpha', 0, 10),
         'gamma':hp.uniform('gamma', 0, 10),
         'reg_lambda':hp.uniform('reg_lambda', 0, 10),
         'n_estimators': hp.quniform('n_estimators', 1, 1000, 1),
         'fp_length': hp.choice('fp_length', fp_length_list)
}

import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import r2_score
from scipy.stats import linregress

def make_plot(total_data, range, tick):

  X_title = 'Real removal (%)'
  Y_title = 'Pred removal (%)'

  title_font_size = 43 # 33
  tickfont_size = 35 # 23
  ticklen = 5 # 5
  tickwidth = 2 # 2
  title_standoff = 20

  plot_width = 610 # 700
  plot_height = 500 # 500

  fig = px.scatter(total_data, x='y_true', y='y_pred', color = 'mark',
                  width=plot_width, height=plot_height, range_x=[-5, range], range_y=[-5, range]
                  )

  fig.update_layout(
      xaxis = dict(
          dtick=tick,
          title_text = X_title,
          title_font = {"size": title_font_size},
          tickfont = dict(family='Arial', color='black', size=tickfont_size),
          title_standoff = title_standoff,
          color='black',
          ticks="inside",
          ticklen=ticklen,
          tickwidth=tickwidth,
          title_font_family="Arial"),
      yaxis = dict(
          dtick = tick,
          title_text = Y_title,
          title_font = {"size": title_font_size},
          tickfont = dict(family='Arial', color='black', size=tickfont_size),
          title_standoff = title_standoff,
          color='black',
          ticks="inside",
          ticklen=ticklen,
          tickwidth=tickwidth,
          title_font_family="Arial"))

  fig.update_layout(plot_bgcolor='rgb(256,256,256)', showlegend=True)

  fig.update_xaxes(showline=True, linewidth=4, linecolor='black', mirror=True)
  fig.update_yaxes(showline=True, linewidth=4, linecolor='black', mirror=True)

  random_x = [0, 100]
  random_y0 = [0, 100]

  fig.add_trace(go.Scatter(x=random_x, y=random_y0, line=dict(color='red', width=2), marker=dict(size=1)))

  fig.update_traces(marker=dict(size=10, opacity=0.65, line=dict(width=0.7)), selector=dict(mode='markers'))

  fig.show()

def hyp_opt(objective):
  fp_length_list = [2048]

  space = {
          'max_delta_step': hp.uniform('max_delta_step', 0, 10),
          'learning_rate': hp.uniform('learning_rate', 0.001, 0.03),
          'max_depth': hp.quniform('max_depth', 1,10,1),
          'min_child_weight': hp.uniform('min_child_weight', 1,10),
          'subsample': hp.uniform('subsample', 0.5, 1.0),
          'reg_alpha':hp.uniform('reg_alpha', 0, 10),
          'gamma':hp.uniform('gamma', 0, 10),
          'reg_lambda':hp.uniform('reg_lambda', 0, 10),
          'n_estimators': hp.quniform('n_estimators', 1, 500, 1),
          'fp_length': hp.choice('fp_length', fp_length_list)
  }

  out_file ='per.csv'
  off_connection =open(out_file, 'w')
  writer = csv.writer(off_connection)
  writer.writerow(['loss','train_loss', 'params', 'iteration'])
  off_connection.close()
  tpe_algo = tpe.suggest
  bayes_trial = Trials()

  global ITERATION
  ITERATION = 0

  tpe_algo = tpe.suggest
  bayes_trial = Trials()

  best = fmin(fn=objective, space=space, algo=tpe_algo, trials=bayes_trial, max_evals=50, rstate=np.random.default_rng(np.random.randint(0,50)))
  best['fp_length'] = int(fp_length_list[best['fp_length']])

  return best

In [ ]:
from datetime import datetime
import pytz

now_utc = datetime.now(pytz.utc)
est_timezone = pytz.timezone('US/Eastern')
now_est = now_utc.astimezone(est_timezone)
date = now_est.strftime('%Y%m%d_%H:%M:%S')

title_rej = 'rej_prediction_XGBoost'

directory_rej = '/gdrive/My Drive/ML-BO_followup/prediction_result/' + date + '_' + title_rej + '/'
createFolder(directory_rej)

columns_removed = ['Pure_water_flux, LMH/bar', 'DOI', 'cation','Additive X2 type in aqueous phase',
                   'Additive X2 concentration, wt%', 'Transmembrane pressure, bar', 'anion', 'Aqueous phase pH']


df = pd.read_excel('/gdrive/My Drive/Li-pore_size/ML-BO_data_Li.xlsx', sheet_name='Datasets')
df.drop(columns_removed, axis=1, inplace=True)

df.columns = ['Monomer A1 type', 'a1-smile', 'Monomer A2 type', 'a2-smile',
              'A1/A2 ratio', 'Monomer A concentration', 'Monomer B type',
              'b-smile', 'Monomer B concentration', 'Organic solvent type',
              'Additive X1 in aqueous', 'Additive X1 concentration',
               'Additive Y type in organic  phase',
              'Additive Y concentration', 'Heat curing time',
              'Heat curing termperature',
              'salt_concentration', 'cat_val', 'cat_r', 'cat_hyd_r', 'cat_hyd_e',
              'an_val', 'an_r', 'an_hyd_r', 'an_hyd_e', 'Polymerization time',
              'Permeability', 'salt_rejection', 'Done']

categorical_features = ['Organic solvent type', 'Additive X1 in aqueous']

numeric_features = ['A1/A2 ratio', 'Monomer A concentration', 'Monomer B concentration', 'Additive X1 concentration',
                    'Additive Y concentration', 'Heat curing time', 'Heat curing termperature',
                    'salt_concentration', 'cat_val', 'cat_r', 'cat_hyd_r', 'cat_hyd_e', 'an_val', 'an_r', 'an_hyd_r',
                    'an_hyd_e', 'Polymerization time']

df[numeric_features] = df[numeric_features].replace('-', np.nan)

df['a2-smile'] =  df['a2-smile'].replace('-', 'C')

df['Monomer A concentration'] = pd.to_numeric(df['Monomer A concentration'], errors='coerce')
df['salt_rejection'] = pd.to_numeric(df['salt_rejection'], errors='coerce')

df = df[df['a1-smile'] != 'xxx']
df = df[df['a2-smile'] != 'xxx']
df = df[df['b-smile'] != 'xxx']
df = df[df['Monomer A1 type'] != 'Potassium 2,5-dihydroxybenzenesulfonate']

df['a1-smile'] = df['a1-smile'].fillna('C')
df['a2-smile'] = df['a2-smile'].fillna('C')
df['b-smile'] = df['b-smile'].fillna('C')

total_df = df[df['Done']==1]
total_df.reset_index(drop=True, inplace=True)
total_df.drop(columns=['Done', 'Monomer B type'], inplace=True)

total_df['salt_rejection'] = total_df['salt_rejection'].replace('-', np.nan)
total_df['Permeability'] = total_df['Permeability'].replace('-', np.nan)
total_df['a2-smile'] = total_df['a2-smile'].replace('-', np.nan)

total_rej = total_df.dropna(subset=['salt_rejection'])
y_rej = total_rej['salt_rejection']
total_rej = total_rej.drop(columns=['Permeability', 'salt_rejection'])

total_per = total_df.dropna(subset=['Permeability'])
y_per = total_per['Permeability']
total_per = total_per.drop(columns=['Permeability', 'salt_rejection'])

X_train_rej, X_test_rej, y_train_rej, y_test_rej = train_test_split(total_rej, y_rej, test_size=0.2, random_state=42)
X_train_rej.reset_index(drop=True, inplace=True)
X_test_rej.reset_index(drop=True, inplace=True)

X_train_per, X_test_per, y_train_per, y_test_per = train_test_split(total_per, y_per, test_size=0.2, random_state=42)
X_train_per.reset_index(drop=True, inplace=True)
X_test_per.reset_index(drop=True, inplace=True)
y_train_rej.reset_index(drop=True, inplace=True)
y_test_rej.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.model_selection import cross_validate, ShuffleSplit, KFold

categorical_features_per = categorical_features.copy()
numeric_features_per = numeric_features.copy()

en=ce.backward_difference.BackwardDifferenceEncoder(cols = categorical_features)

train_data_rej, val_data_rej = data_split(X_train_rej, 10, 0.2, 10)
train_data_rej.reset_index(drop=True, inplace=True)
val_data_rej.reset_index(drop=True, inplace=True)

fp_rej = morgan_fp(0, 2048)
kf = KFold(shuffle=True, random_state=10)
sc_rej = PowerTransformer()

y_train_rej.reset_index(drop=True, inplace=True)
y_test_rej.reset_index(drop=True, inplace=True)

In [ ]:
import ast
from rdkit import DataStructs, Chem
from rdkit.Chem import AllChem

params_rej = {'fp_length': 2048,
              'gamma': 1.5412511395696913,
              'learning_rate': 0.08103909978496358,
              'max_delta_step': 6.29569181619016,
              'max_depth': 6.0,
              'min_child_weight': 7.212247399661489,
              'n_estimators': 727.0,
              'reg_alpha': 3.816910539411469,
              'reg_lambda': 4.238030816945888,
              'subsample': 0.6034272913431271}

fp_rej = morgan_fp(0, params_rej['fp_length'])

en_rej=ce.backward_difference.BackwardDifferenceEncoder(cols = categorical_features)


x_train_rej = conv_data_pd(X_train_rej, X_train_rej, fp_rej, en_rej, sc_rej, numeric_features, 'rej')
x_test_rej = conv_data_pd(X_test_rej, X_train_rej, fp_rej, en_rej, sc_rej, numeric_features, 'rej')

model_rej = XGBRegressor(max_delta_step=params_rej['max_delta_step'], learning_rate = params_rej['learning_rate'],
                    max_depth = int(params_rej['max_depth']), min_child_weight= params_rej['min_child_weight'],
                    subsample=params_rej['subsample'],reg_alpha=params_rej['reg_alpha'],gamma= params_rej['gamma'],
                    reg_lambda= params_rej['reg_lambda'],n_estimators=int(params_rej['n_estimators']), random_state=10)
model_rej.fit(x_train_rej, y_train_rej)

y_train_rej.reset_index(drop=True, inplace=True)
y_test_rej.reset_index(drop=True, inplace=True)

np.round(r2_score(y_train_rej, model_rej.predict(x_train_rej)), 2)

0.96

In [ ]:
train_data_per, val_data_per = data_split(X_train_per, 10, 0.2, 10)
train_data_per.reset_index(drop=True, inplace=True)
val_data_per.reset_index(drop=True, inplace=True)

fp_per = morgan_fp(0, 2048)
kf = KFold(shuffle=True, random_state=10)
sc_per = PowerTransformer()

y_train_per.reset_index(drop=True, inplace=True)
y_test_per.reset_index(drop=True, inplace=True)

In [ ]:
params_per = {'fp_length': 2048,
              'gamma': 0.5370309939072397,
              'learning_rate': 0.05144141844082313,
              'max_delta_step': 6.427683378298868,
              'max_depth': 10.0,
              'min_child_weight': 9.384698920602181,
              'n_estimators': 523.0,
              'reg_alpha': 6.7864101789423055,
              'reg_lambda': 3.1001740844985735,
              'subsample': 0.6699461278318496}

categorical_features = ['Organic solvent type', 'Additive X1 in aqueous']

numeric_features = ['A1/A2 ratio', 'Monomer A concentration', 'Monomer B concentration', 'Additive X1 concentration',
                    'Additive Y concentration', 'Heat curing time', 'Heat curing termperature',
                    'salt_concentration', 'cat_val', 'cat_r', 'cat_hyd_r', 'cat_hyd_e', 'an_val', 'an_r', 'an_hyd_r',
                    'an_hyd_e', 'Polymerization time']

fp_per = morgan_fp(0, params_per['fp_length'])
# sc_per = PowerTransformer()
en_per=ce.backward_difference.BackwardDifferenceEncoder(cols = categorical_features)

x_train_per = conv_data_pd(X_train_per, X_train_per, fp_per, en_per, sc_per, numeric_features, 'per')
model_per = XGBRegressor(max_delta_step=params_per['max_delta_step'], learning_rate = params_per['learning_rate'],
                    max_depth = int(params_per['max_depth']), min_child_weight= params_per['min_child_weight'],
                    subsample=params_per['subsample'],reg_alpha=params_per['reg_alpha'],gamma= params_per['gamma'],
                    reg_lambda= params_per['reg_lambda'],n_estimators=int(params_per['n_estimators']), random_state=10)
model_per.fit(x_train_per, y_train_per)

y_train_per_pred = model_per.predict(x_train_per)

np.round(r2_score(y_train_per, y_train_per_pred), 2)

0.82

In [ ]:
file_directory = ''

In [ ]:
mom_l = pd.read_csv(file_directory + 'mom_limit_rej_per_0521.csv')
mom_l.drop(columns=['sim'], inplace=True)

mom_l.loc[79, 'a-smile']='C'
mom_l.loc[79, 'a-mw']=0
mom_l = mom_l.rename(columns={'Monomer name': 'Monomer A1 type'})
mom_l.loc[79, 'Monomer A1 type']='Carbon'

In [ ]:
MB = df[['Monomer B type', 'b-smile']].drop_duplicates().reset_index(drop=True)
MB = MB[MB['b-smile']!='xxx']
MB.dropna(axis=0, inplace=True)

OB = df[['Organic solvent type']].drop_duplicates().reset_index(drop=True)
X1T = df[['Additive X1 in aqueous']].drop_duplicates().reset_index(drop=True)
# X2T = df[['Additive X2 type in aqueous phase']].drop_duplicates().reset_index(drop=True)
YT = df[['Additive Y type in organic  phase']].drop_duplicates().reset_index(drop=True)

In [ ]:
###optimization

col = list(train_data_rej.columns)
# col.append('permeabiility')

space_opt = {'a1-smile': hp.randint('a1-smile', len(mom_l)),
             'a2-smile': hp.randint('a2-smile', len(mom_l)),
              # 'Monomer B type': hp.randint('Monomer B type', len(MB)),
             'b-smile': hp.randint('b-smile', len(MB)),
             'Organic solvent type': hp.randint('Organic solvent type', len(OB)),
             'Additive X1 in aqueous': hp.randint('Additive X1 in aqueous', len(X1T)),
             'Additive Y type in organic  phase': hp.randint('Additive Y type in organic  phase', len(YT)),
            #  'Additive X2 type in aqueous phase': hp.randint(' Additive X2 type in aqueous phase', len(X2T)),
             'A1/A2 ratio': hp.uniform('A1/A2 ratio', 0, 10),
             'Monomer A concentration': hp.uniform('Monomer A concentration', 0.01, 3),
             'Monomer B concentration': hp.uniform('Monomer B concentration', 0.01, 1),

             'Additive X1 concentration':hp.choice('Additive X1 concentration', [hp.uniform('Additive X1 concentration range', 0, 1), np.nan]),
            #  'Additive X2 concentration':hp.choice('Additive X2 concentration', [hp.uniform('Additive X2 concentration range', 0, 1.5), np.nan]),
             'Additive Y concentration':hp.choice('Additive Y concentration', [hp.uniform('Additive Y concentration range', 0, 1), np.nan]),

            #  'Aqueous phase pH':hp.choice('Aqueous phase pH', [hp.uniform('pH range', 7, 12), np.nan]),
             'Polymerization time':hp.quniform('Polymerization time_a', 5, 600, 1),
             'Heat curing time':hp.quniform('Heat curing time', 1, 20,1),
             'Heat curing temperature': hp.quniform('Heat curing temperature', 22, 70,1),
            #  'Transmembrane pressure':hp.quniform('Transmembrane pressure', 2, 10,1)
            }

In [ ]:
##Na2SO4

def get_x_l(params):
    hh = pd.DataFrame(columns=col)
    hh.loc[0, 'Monomer A1 type']=mom_l.loc[params['a1-smile'],'Monomer A1 type']
    hh.loc[0, 'Monomer A2 type']=mom_l.loc[params['a2-smile'],'Monomer A1 type']
    hh.loc[0, 'a1-smile']=mom_l.loc[params['a1-smile'],'a-smile']
    hh.loc[0, 'a2-smile']=mom_l.loc[params['a2-smile'],'a-smile']
    hh.loc[0, 'b-smile']=MB.loc[params['b-smile'],'b-smile']
    hh.loc[0, 'A1/A2 ratio']=params['A1/A2 ratio']
    hh.loc[0, 'Monomer A concentration']=params['Monomer A concentration']
    hh.loc[0, 'Monomer B concentration']=params['Monomer B concentration']

    hh.loc[0, 'Additive Y type in organic  phase']=params['Additive Y type in organic  phase']
    # hh.loc[0, 'Additive X2 type in aqueous phase']=params['Additive X2 type in aqueous phase']

    hh.loc[0, 'Additive X1 concentration']=params['Additive X1 concentration']
    # hh.loc[0, 'Additive X2 concentration']=params['Additive X2 concentration']
    hh.loc[0, 'Additive Y concentration']=params['Additive Y concentration']

    # hh.loc[0, 'Aqueous phase pH']=params['Aqueous phase pH']
    hh.loc[0, 'Polymerization time']=params['Polymerization time']
    hh.loc[0, 'Heat curing time']=params['Heat curing time']
    hh.loc[0, 'Heat curing temperature']=params['Heat curing temperature']
    # hh.loc[0, 'Transmembrane pressure']=params['Transmembrane pressure']

    hh.loc[0, 'Organic solvent type']= OB.loc[params['Organic solvent type'], 'Organic solvent type']
    hh.loc[0, 'Additive X1 in aqueous']= X1T.loc[params['Additive X1 in aqueous'], 'Additive X1 in aqueous']

    hh.loc[0, 'cat_val']=1
    hh.loc[0, 'cat_r']=0.102
    hh.loc[0, 'cat_hyd_r' ]=0.358
    hh.loc[0, 'cat_hyd_e' ]=-365

    hh.loc[0, 'an_val']=-2
    hh.loc[0, 'an_r']=0.23
    hh.loc[0, 'an_hyd_r' ]=0.379
    hh.loc[0, 'an_hyd_e' ]=-1080
    hh.loc[0, 'salt_concentration']=1000
    # hh.loc[0, 'salt_rejection']=np.nan
    # hh.loc[0, 'Permeability']=np.nan

    return hh

In [ ]:
from hyperopt.pyll.stochastic import sample
hh = sample(space_opt)
d = get_x_l(hh)

In [ ]:
target_perm = 10

def fit(params):
    hh = get_x_l(params)
    if hh['a1-smile'][0] == hh['a2-smile'][0]:
        hh.loc[0, 'a2-smile'] = 'C'
        hh.loc[0, 'a2-mw'] = 0
    for i in hh[hh['Additive X1 in aqueous'].isnull() == True].index:
        hh.loc[i, 'Additive X1 concentration'] = 0
    if hh['b-smile'][0] == 'C':
        hh.loc[0, 'A1/A2 ratio'] = 0

    x_salt = conv_data_pd_opt(hh, X_train_rej, fp_rej, en_rej, sc_rej, numeric_features, 'rej')
    x_per = conv_data_pd_opt(hh, X_train_per, fp_per, en_per, sc_per, numeric_features, 'per')

    # Ensure numeric features are of type float
    x_salt[numeric_features] = x_salt[numeric_features].astype(float)
    x_per[numeric_features] = x_per[numeric_features].astype(float)

    x_salt.to_csv('x_salt.csv')

    pred = model_rej.predict(x_salt)[0]
    per_pred = model_per.predict(x_per)[0]

    return abs((100 - pred)/per_pred) + abs(per_pred - target_perm), (100 - pred)/per_pred, pred, per_pred


def objective(params):
    global ITERATION
    ITERATION +=1

    loss,y, salt, per = fit(params)

    off_connection = open(out_file, 'a')
    writer = csv.writer(off_connection)
    writer.writerow([loss,y, salt, per, params, ITERATION])
    return {'loss':loss,'Y':y,'salt':salt, 'per':per, 'params': params, 'iteration':ITERATION, 'status':STATUS_OK}

import csv
out_file = '/gdrive/My Drive/ML-BO_followup/' + 'salt_opt_l10_withnm.csv'
off_connection =open(out_file, 'w')
writer = csv.writer(off_connection)
writer.writerow(['loss','Y','salt', 'per', 'params', 'iteration'])
off_connection.close()

tpe_algo = tpe.suggest
bayes_trial = Trials()

In [ ]:
#%%capture
global ITERATION
ITERATION = 0
best = fmin(fn = objective, space=space_opt, algo = tpe_algo, trials = bayes_trial, max_evals=10000, rstate=np.random.default_rng(np.random.randint(0,50)))

 37%|███▋      | 3704/10000 [1:31:19<2:49:10,  1.61s/trial, best loss: 0.020843031786254976]